In [ ]:

---

# 🟦 **resume_matcher.py**

This is a **full end-to-end LLM-powered resume screening system**.

```python
# ==================================================
# LLM-BASED RESUME SCREENING & CANDIDATE MATCHING
# ==================================================

import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openai

# -------------------------------
# 1. Load Embedding Model
# -------------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

# -------------------------------
# 2. Load Job Description
# -------------------------------
with open("job_description.txt", "r", encoding="utf-8") as f:
    job_description = f.read()

# -------------------------------
# 3. Load Resumes
# -------------------------------
resume_folder = "resumes/"
resumes = []
names = []

for file in os.listdir(resume_folder):
    if file.endswith(".txt"):
        with open(os.path.join(resume_folder, file), "r", encoding="utf-8") as f:
            resumes.append(f.read())
            names.append(file.replace(".txt", ""))

# -------------------------------
# 4. Generate Embeddings
# -------------------------------
jd_embedding = model.encode([job_description])
resume_embeddings = model.encode(resumes)

# -------------------------------
# 5. Compute Similarity Scores
# -------------------------------
similarities = cosine_similarity(jd_embedding, resume_embeddings)[0]

results = pd.DataFrame({
    "Candidate": names,
    "Similarity_Score": similarities
}).sort_values(by="Similarity_Score", ascending=False)

print("\nCandidate Ranking:")
print(results)

# -------------------------------
# 6. LLM Explanation (Top Candidate)
# -------------------------------
openai.api_key = "YOUR_API_KEY"

top_candidate = results.iloc[0]["Candidate"]
top_resume = resumes[names.index(top_candidate)]

prompt = f"""
You are a hiring assistant.

Job Description:
{job_description}

Candidate Resume:
{top_resume}

Explain in 4–5 lines why this candidate is a good fit for this role.
"""

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role":"user","content":prompt}]
)

print("\nLLM Explanation for Top Candidate:")
print(response["choices"][0]["message"]["content"])

# -------------------------------
# 7. Save Results
# -------------------------------
results.to_csv("candidate_ranking.csv", index=False)
print("\nCandidate ranking saved successfully!")
